# 🔥 한국어 간판 OCR 파인튜닝 가이드

**DeepSeek-OCR 모델을 한국어 간판 이미지로 파인튜닝하는 완전 가이드**

---

## 📚 프로젝트 개요

- **목표**: 한국어 간판 이미지를 정확하게 읽어내는 OCR 모델 만들기
- **모델**: DeepSeek-OCR (3B 파라미터)
- **데이터**: 138,149개 학습 이미지 + 17,272개 검증 이미지
- **방법**: Unsloth + LoRA (메모리 효율적 학습)
- **플랫폼**: Google Colab Pro (A100 GPU 권장)

---

## 🎯 전체 과정 요약

### **STEP 1: 로컬 (WSL)에서 데이터 준비** ⬅️ 먼저 이것부터!
```bash
# 데이터 압축 (WSL 터미널에서 실행)
cd /root/data/deepseek_ocr
tar -czf korean_signboard_data.tar.gz train.jsonl val.jsonl filtered_train/images filtered_val/images
```

### **STEP 2: Google Drive에 업로드**
- 생성된 `korean_signboard_data.tar.gz` 파일을 Google Drive에 업로드
- 위치: `내 드라이브` (루트)에 바로 업로드

### **STEP 3: Google Colab에서 학습** ⬅️ 이 노트북!
- 이 노트북을 Google Colab에 업로드
- **런타임 → 런타임 유형 변경 → A100 GPU 선택**
- 셀을 순서대로 실행

---

## ⏱️ 예상 소요 시간 (A100 GPU)

| 단계 | 예상 시간 |
|------|----------|
| 환경 설정 | 3-5분 |
| 데이터 압축 해제 | 2-3분 |
| 모델 로딩 | 1-2분 |
| **전체 학습 (138K 샘플)** | **2-3시간** |
| 평가 및 저장 | 5-10분 |

💡 **Tip**: 처음에는 `max_steps=100`로 테스트 후 전체 학습 진행 권장

---

## ⚠️ 시작하기 전 체크리스트

- [ ] WSL에서 데이터 압축 완료
- [ ] Google Drive에 `korean_signboard_data.tar.gz` 업로드 완료
- [ ] Google Colab Pro 가입 (A100 GPU 사용 위해)
- [ ] 이 노트북을 Colab에 업로드
- [ ] **런타임 유형을 A100 GPU로 변경**

---

## 🚀 지금부터 시작!

아래 셀들을 **순서대로** 실행하세요. 각 셀은 자동으로 다음 단계를 준비합니다.

## 1️⃣ 환경 설정 및 GPU 확인

**✅ 여기서 확인할 것:**
1. GPU가 A100인지 확인 (상단에 `Tesla A100-SXM4-40GB` 표시되어야 함)
2. 메모리가 40GB 이상인지 확인

**⚠️ 만약 A100이 아니라면:**
- 상단 메뉴: **런타임 → 런타임 유형 변경 → A100 GPU** 선택 후 다시 실행

In [ ]:
# GPU 확인
print("🔍 GPU 정보 확인 중...\n")
!nvidia-smi

print("\n" + "="*60)
print("✅ A100 GPU가 보이면 정상입니다!")
print("⚠️  T4나 다른 GPU라면 런타임 유형을 A100으로 변경하세요")
print("="*60)

# Unsloth 설치
print("\n📦 Unsloth 및 필요한 라이브러리 설치 중... (3-5분 소요)")
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps "trl<0.9.0" peft accelerate bitsandbytes

print("\n✅ 설치 완료!")

## 2️⃣ Google Drive 마운트 및 데이터 다운로드

**📋 사전 준비사항:**

### 로컬 (WSL)에서 이미 완료했어야 할 것:
```bash
cd /root/data/deepseek_ocr
tar -czf korean_signboard_data.tar.gz train.jsonl val.jsonl filtered_train/images filtered_val/images
```

### Google Drive에 업로드:
- 생성된 `korean_signboard_data.tar.gz` 파일을 **내 드라이브 루트**에 업로드

---

**✅ 준비되었다면 아래 셀 실행:**
- Google 계정 로그인 팝업이 뜹니다
- 권한 허용을 클릭하세요

In [ ]:
from google.colab import drive
import os

print("🔐 Google Drive 마운트 중...")
print("➡️  팝업창에서 Google 계정을 선택하고 권한을 허용하세요\n")
drive.mount('/content/drive')

print("\n✅ Drive 마운트 완료!")

# 현재 디스크 상태 확인
print("\n📊 압축 해제 전 디스크 상태:")
!df -h /content

# 데이터 압축 해제
print("\n📦 데이터 압축 파일 확인 중...")

# 파일 경로 확인 (여러 가능한 위치 체크)
possible_paths = [
    "/content/drive/MyDrive/korean_signboard_data.tar.gz",
    "/content/drive/My Drive/korean_signboard_data.tar.gz",
]

tar_file = None
for path in possible_paths:
    if os.path.exists(path):
        tar_file = path
        file_size = os.path.getsize(path) / (1024**3)
        print(f"✅ 파일 발견: {path}")
        print(f"📏 파일 크기: {file_size:.2f}GB")
        break

if tar_file is None:
    print("❌ 데이터 파일을 찾을 수 없습니다!")
    print("\n📍 파일 위치 확인:")
    !ls -lh /content/drive/MyDrive/ | head -20
    print("\n⚠️  korean_signboard_data.tar.gz 파일을 Google Drive 루트에 업로드했는지 확인하세요!")
else:
    # 압축 해제
    print(f"\n📂 데이터 압축 해제 시작...")
    print(f"⏱️  예상 시간: 5-10분")
    print(f"💡 압축 해제 완료 즉시 압축 파일을 삭제하면 용량 확보됩니다!\n")
    
    !mkdir -p /content/data
    !tar -xzf "{tar_file}" -C /content/data/
    
    print("\n✅ 압축 해제 완료!")
    
    # 압축 해제 후 디스크 상태
    print("\n📊 압축 해제 후 디스크 상태:")
    !df -h /content
    
    print("\n📊 데이터 확인:")
    !ls -lh /content/data/
    
    # 샘플 수 확인
    import subprocess
    try:
        train_count = int(subprocess.check_output("wc -l /content/data/train.jsonl", shell=True).decode().split()[0])
        val_count = int(subprocess.check_output("wc -l /content/data/val.jsonl", shell=True).decode().split()[0])
        
        print(f"\n📈 데이터셋 크기:")
        print(f"  - 학습 샘플: {train_count:,}개")
        print(f"  - 검증 샘플: {val_count:,}개")
        
        print(f"\n💡 용량 부족 시 Drive의 압축 파일을 삭제하세요:")
        print(f"   !rm {tar_file}")
    except Exception as e:
        print(f"⚠️  샘플 수 확인 중 에러: {e}")

In [ ]:
# 🔄 Colab 초기화 (압축 해제 실패 시)
import shutil
import os

print("🗑️  기존 데이터 삭제 중...")

# /content/data 삭제
if os.path.exists('/content/data'):
    shutil.rmtree('/content/data')
    print("✅ /content/data 삭제 완료")

# 압축 파일 삭제 (Colab 로컬에 있다면)
if os.path.exists('/content/korean_signboard_data.tar.gz'):
    os.remove('/content/korean_signboard_data.tar.gz')
    print("✅ 압축 파일 삭제 완료")

# 체크포인트 삭제 (있다면)
if os.path.exists('/content/deepseek_ocr_korean_signboard'):
    shutil.rmtree('/content/deepseek_ocr_korean_signboard')
    print("✅ 체크포인트 삭제 완료")

print("\n📊 현재 디스크 사용량:")
!df -h /content

print("\n✅ 초기화 완료! 아래 셀부터 다시 실행하세요.")

### ⚠️ 압축 해제 실패 시 초기화

**압축 해제가 실패했다면 아래 셀을 먼저 실행하세요:**

## 3️⃣ DeepSeek-OCR 모델 로딩

**🤖 여기서 하는 일:**
- DeepSeek-OCR 3B 모델을 Hugging Face에서 다운로드
- 4비트 양자화로 메모리 효율적 로딩 (8GB → 2GB)
- 약 1-2분 소요

**💡 Tip**: 처음 실행 시 모델 다운로드로 시간이 조금 걸립니다

In [ ]:
from unsloth import FastVisionModel
from datasets import load_dataset
import torch

print("📥 DeepSeek-OCR 모델 다운로드 및 로딩 중...")
print("(처음 실행 시 모델 다운로드로 1-2분 소요될 수 있습니다)\n")

# 최대 시퀀스 길이
max_seq_length = 2048

# 4비트 양자화로 모델 로드
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/DeepSeek-OCR",
    load_in_4bit=True,  # 메모리 절약: 8GB → 2GB
    use_gradient_checkpointing="unsloth",  # Unsloth 최적화
)

print("✅ 모델 로딩 완료!")
print(f"📊 모델 정보:")
print(f"  - 이름: DeepSeek-OCR")
print(f"  - 파라미터: ~3B")
print(f"  - 양자화: 4-bit")
print(f"  - 최대 길이: {max_seq_length} 토큰")

In [ ]:
# LoRA 적용
print("🔧 LoRA 어댑터 설정 중...")
print("(Vision + Language 레이어 모두 파인튜닝)\n")

model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers=True,      # Vision 인코더 학습
    finetune_language_layers=True,    # Language 모델 학습
    finetune_attention_modules=True,  # Attention 레이어 학습
    finetune_mlp_modules=True,        # MLP 레이어 학습
    
    r=16,              # LoRA rank (높을수록 표현력 증가)
    lora_alpha=16,     # LoRA scaling
    lora_dropout=0,    # Dropout 없음
    bias="none",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

# 학습 가능한 파라미터 확인
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print("✅ LoRA 적용 완료!")
print(f"\n📊 파라미터 정보:")
print(f"  - 전체 파라미터: {total_params:,}")
print(f"  - 학습 파라미터: {trainable_params:,}")
print(f"  - 학습 비율: {trainable_params/total_params*100:.2f}%")
print(f"\n💡 LoRA를 사용하면 전체의 {trainable_params/total_params*100:.2f}%만 학습하여 메모리와 시간을 절약합니다!")

## 4️⃣ 데이터셋 로딩 및 확인

**📊 데이터 형식:**
- `train.jsonl`: 138,149개 학습 샘플
- `val.jsonl`: 17,272개 검증 샘플
- 각 샘플: 이미지 경로 + 대화 형식 (질문 → 답변)

**✅ 여기서 확인할 것:**
- 샘플 수가 정확한지 확인
- 샘플 데이터 구조 확인

In [ ]:
print("📂 데이터셋 로딩 중...\n")

# 데이터셋 로드
train_dataset = load_dataset(
    "json", 
    data_files="/content/data/train.jsonl", 
    split="train"
)

val_dataset = load_dataset(
    "json", 
    data_files="/content/data/val.jsonl", 
    split="train"
)

print("✅ 데이터셋 로딩 완료!")
print(f"\n📊 데이터셋 크기:")
print(f"  - 학습 샘플: {len(train_dataset):,}개")
print(f"  - 검증 샘플: {len(val_dataset):,}개")
print(f"  - 전체: {len(train_dataset) + len(val_dataset):,}개")

# 샘플 확인
print("\n" + "="*60)
print("📄 샘플 데이터 구조 (첫 번째 샘플):")
print("="*60)
sample = train_dataset[0]
print(f"이미지 경로: {sample['image']}")
print(f"\n질문: {sample['conversations'][0]['content']}")
print(f"답변: {sample['conversations'][1]['content']}")
print("="*60)

print(f"\n💡 A100 GPU 기준 예상 학습 시간: 2-3시간")

## 5️⃣ 트레이너 설정

**⚙️ 학습 설정:**
- **배치 크기**: 2 (per device)
- **Gradient Accumulation**: 4 (효과적 배치 크기 = 8)
- **학습 에폭**: 1회 전체 데이터
- **학습률**: 2e-4
- **최적화**: AdamW 8-bit

**💾 저장 설정:**
- 500 스텝마다 체크포인트 저장
- 최대 3개 체크포인트 유지
- 100 스텝마다 검증

**⏱️ 예상 시간:**
- Total Steps: ~17,268 (138,149 / 8)
- A100 기준: **2-3시간**

In [ ]:
from transformers import TextStreamer
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

print("⚙️  트레이너 설정 중...\n")

# 데이터 콜레이터
data_collator = UnslothVisionDataCollator(model, tokenizer)

# 학습 설정값
batch_size = 2
grad_accum = 4
effective_batch_size = batch_size * grad_accum
total_steps = len(train_dataset) // effective_batch_size

print(f"📊 학습 설정:")
print(f"  - 배치 크기: {batch_size}")
print(f"  - Gradient Accumulation: {grad_accum}")
print(f"  - 효과적 배치 크기: {effective_batch_size}")
print(f"  - 전체 스텝 수: {total_steps:,}")
print(f"  - 학습 에폭: 1")
print(f"  - 학습률: 2e-4")
print(f"\n⏱️  A100 GPU 기준 예상 시간: 2-3시간")
print(f"\n💡 중간에 중단되어도 체크포인트에서 재개 가능합니다!")

# 트레이너 설정
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=SFTConfig(
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum,
        
        warmup_steps=100,
        num_train_epochs=1,  # 1 에폭 (전체 데이터 1회)
        # max_steps=100,  # 테스트용: 주석 해제하면 100 스텝만 학습
        
        learning_rate=2e-4,
        fp16=not is_bf16_supported(),
        bf16=is_bf16_supported(),
        
        logging_steps=25,      # 25 스텝마다 로그
        eval_steps=500,        # 500 스텝마다 검증
        save_steps=500,        # 500 스텝마다 저장
        save_total_limit=3,    # 최대 3개 체크포인트
        
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        
        output_dir="/content/deepseek_ocr_korean_signboard",
        report_to="none",
        
        remove_unused_columns=False,
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
        dataset_num_proc=4,
        max_seq_length=max_seq_length,
    ),
)

print("\n✅ 트레이너 설정 완료!")
print("\n🚀 다음 셀을 실행하면 학습이 시작됩니다!")

In [ ]:
# 🚀🚀🚀 학습 시작! 🚀🚀🚀
import time

print("="*60)
print("🚀 학습 시작!")
print("="*60)
print(f"\n📊 학습 정보:")
print(f"  - 학습 샘플: {len(train_dataset):,}개")
print(f"  - 검증 샘플: {len(val_dataset):,}개")
print(f"  - Effective Batch Size: {2 * 4} (batch_size × grad_accum)")
print(f"  - Total Steps: {len(train_dataset) // (2 * 4):,}")
print(f"\n⏱️  예상 시간: 2-3시간 (A100 GPU)")
print(f"\n💡 팁:")
print(f"  - 로그를 보면서 loss가 감소하는지 확인하세요")
print(f"  - 500 스텝마다 자동으로 체크포인트가 저장됩니다")
print(f"  - 중단되어도 체크포인트에서 재개 가능합니다")
print("\n" + "="*60)
print("학습 시작...\n")

start_time = time.time()

# 학습 실행
trainer_stats = trainer.train()

end_time = time.time()
elapsed_time = end_time - start_time

print("\n" + "="*60)
print("🎉 학습 완료!")
print("="*60)
print(f"\n⏱️  총 소요 시간: {elapsed_time/3600:.2f}시간 ({elapsed_time/60:.1f}분)")
print(f"📊 최종 Loss: {trainer_stats.training_loss:.4f}")
print("\n✅ 다음 단계: 모델 저장 및 평가")

## 6️⃣ 모델 저장

**💾 저장 위치:**
1. **Colab 로컬**: `/content/deepseek_ocr_korean_signboard/final`
   - Colab 세션이 끝나면 삭제됨
   
2. **Google Drive**: `/content/drive/MyDrive/deepseek_ocr_finetuned`
   - 영구 보관 (중요!)
   - 나중에 로컬로 다운로드 가능

**⚠️ 중요**: Google Drive 백업을 꼭 확인하세요!

In [ ]:
print("💾 모델 저장 중...\n")

# 로컬 저장
print("1️⃣ Colab 로컬에 저장 중...")
model.save_pretrained("/content/deepseek_ocr_korean_signboard/final")
tokenizer.save_pretrained("/content/deepseek_ocr_korean_signboard/final")
print("✅ 로컬 저장 완료: /content/deepseek_ocr_korean_signboard/final")

# Google Drive에 백업
print("\n2️⃣ Google Drive에 백업 중... (중요!)")
!cp -r /content/deepseek_ocr_korean_signboard/final /content/drive/MyDrive/deepseek_ocr_finetuned
print("✅ Google Drive 백업 완료!")

# 저장된 파일 확인
print("\n📂 저장된 파일:")
!ls -lh /content/drive/MyDrive/deepseek_ocr_finetuned/

print("\n" + "="*60)
print("✅ 모델 저장 완료!")
print("="*60)
print("\n📍 저장 위치:")
print("  - Colab 로컬: /content/deepseek_ocr_korean_signboard/final")
print("  - Google Drive: 내 드라이브/deepseek_ocr_finetuned")
print("\n💡 Google Drive에서 언제든지 다운로드 가능합니다!")
print("="*60)

## 7️⃣ 추론 테스트

**🧪 여기서 하는 일:**
- 학습된 모델로 실제 이미지 테스트
- 간판 이미지 → 텍스트 추출

**💡 다른 이미지로 테스트하려면:**
- `test_image_path` 변수를 원하는 이미지 경로로 변경

In [ ]:
from PIL import Image

print("🧪 추론 테스트 시작...\n")

# 추론 모드로 전환
FastVisionModel.for_inference(model)
print("✅ 추론 모드로 전환 완료")

# 테스트 이미지 로드 (검증 데이터 첫 번째 이미지)
sample = val_dataset[0]
test_image_path = f"/content/data/{sample['image']}"
ground_truth = sample['conversations'][1]['content']

print(f"\n📸 테스트 이미지: {test_image_path}")
print(f"📝 정답: {ground_truth}")

image = Image.open(test_image_path)

# 프롬프트 구성
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "이 간판 이미지의 텍스트를 읽어주세요."}
    ]}
]

# 토큰화
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

# 생성
print("\n🤖 모델 예측:")
print("-" * 40)
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
output = model.generate(
    **inputs, 
    streamer=text_streamer, 
    max_new_tokens=128,
    use_cache=True,
    temperature=0.5,
    min_p=0.1
)
print("-" * 40)

# 결과 비교
predicted_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"\n📊 결과 비교:")
print(f"  정답: {ground_truth}")
print(f"  예측: {predicted_text}")
print(f"  일치: {'✅ 완전 일치!' if predicted_text.strip() == ground_truth.strip() else '❌ 불일치'}")

print("\n✅ 추론 테스트 완료!")

## 8️⃣ 전체 평가 (베이스라인과 비교)

**📊 평가 방법:**
- 검증 데이터 20개 샘플로 테스트
- **완전 일치**: 대소문자/공백까지 정확히 일치
- **정규화 일치**: 소문자 변환 + 공백 제거 후 일치

**🎯 베이스라인 (학습 전) 성능:**
- 완전 일치: 10%
- 정규화 일치: 35%

**✅ 파인튜닝 후 성능 향상 확인!**

In [ ]:
import json
from tqdm import tqdm
import re

# 텍스트 정규화 함수
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', '', text)
    text = re.sub(r'[^가-힣a-z0-9]', '', text)
    return text

print("🔍 전체 평가 시작...\n")
print("📊 20개 샘플로 평가 진행 중...")

# 평가 데이터 로드 (20개 샘플)
eval_samples = val_dataset.select(range(20))

correct = 0
normalized_match = 0
total = 0
results = []

for idx, sample in enumerate(tqdm(eval_samples, desc="평가 중")):
    try:
        # 이미지 로드
        image_path = f"/content/data/{sample['image']}"
        image = Image.open(image_path)
        
        # 정답
        ground_truth = sample['conversations'][1]['content']
        
        # 예측
        messages = [
            {"role": "user", "content": [
                {"type": "image"},
                {"type": "text", "text": sample['conversations'][0]['content']}
            ]}
        ]
        
        input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
        inputs = tokenizer(image, input_text, add_special_tokens=False, return_tensors="pt").to("cuda")
        
        output = model.generate(**inputs, max_new_tokens=128, use_cache=True, temperature=0.5)
        prediction = tokenizer.decode(output[0], skip_special_tokens=True)
        
        # 정확도 계산
        exact_match = prediction.strip() == ground_truth.strip()
        norm_match = normalize_text(prediction) == normalize_text(ground_truth)
        
        if exact_match:
            correct += 1
        
        if norm_match:
            normalized_match += 1
        
        total += 1
        
        # 결과 저장
        results.append({
            "image": sample['image'],
            "ground_truth": ground_truth,
            "prediction": prediction,
            "exact_match": exact_match,
            "normalized_match": norm_match
        })
        
    except Exception as e:
        print(f"에러 (샘플 {idx}): {e}")
        continue

# 결과 출력
print("\n" + "="*60)
print("📊 평가 결과")
print("="*60)
print(f"\n✅ 파인튜닝 후 성능:")
print(f"  - 완전 일치: {correct/total*100:.2f}% ({correct}/{total})")
print(f"  - 정규화 일치: {normalized_match/total*100:.2f}% ({normalized_match}/{total})")

print(f"\n📉 베이스라인 (학습 전) 성능:")
print(f"  - 완전 일치: 10.00% (2/20)")
print(f"  - 정규화 일치: 35.00% (7/20)")

print(f"\n📈 성능 향상:")
print(f"  - 완전 일치: {(correct/total*100 - 10):.2f}% 포인트 증가")
print(f"  - 정규화 일치: {(normalized_match/total*100 - 35):.2f}% 포인트 증가")

print("="*60)

# 틀린 예시 몇 개 출력
print("\n❌ 불일치 샘플 예시:")
wrong_samples = [r for r in results if not r['exact_match']][:3]
for i, sample in enumerate(wrong_samples, 1):
    print(f"\n{i}. {sample['image']}")
    print(f"   정답: {sample['ground_truth']}")
    print(f"   예측: {sample['prediction']}")

print("\n✅ 평가 완료!")

## 9️⃣ 완료 및 다음 단계

**🎉 축하합니다! 모든 과정이 완료되었습니다!**

---

## 📦 저장된 파일 위치:

### Google Drive (영구 보관):
```
내 드라이브/deepseek_ocr_finetuned/
  ├── adapter_config.json
  ├── adapter_model.safetensors
  ├── tokenizer_config.json
  └── ... (기타 파일들)
```

---

## 💻 로컬에서 사용하는 방법:

### 1. Google Drive에서 다운로드
- Google Drive에 접속
- `deepseek_ocr_finetuned` 폴더를 로컬로 다운로드

### 2. 추론 코드 (WSL에서 실행)
```python
from unsloth import FastVisionModel
from PIL import Image

# 모델 로드
model, tokenizer = FastVisionModel.from_pretrained(
    "/path/to/deepseek_ocr_finetuned",  # 다운로드한 경로
    load_in_4bit=True,
)
FastVisionModel.for_inference(model)

# 추론
image = Image.open("signboard.jpg")
messages = [{"role": "user", "content": [
    {"type": "image"},
    {"type": "text", "text": "이 간판 이미지의 텍스트를 읽어주세요."}
]}]

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(image, input_text, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=128)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print(result)
```

---

## 🚀 선택사항:

### HuggingFace Hub에 업로드 (아래 셀 실행):
- 다른 사람과 공유하거나 API로 사용 가능

In [ ]:
print("🎉 모든 작업 완료!")
print("\n" + "="*60)
print("📂 파일 저장 위치")
print("="*60)
print("\n✅ Google Drive (영구 보관):")
print("   📍 내 드라이브/deepseek_ocr_finetuned/")
print("\n✅ Colab 로컬 (세션 종료 시 삭제됨):")
print("   📍 /content/deepseek_ocr_korean_signboard/final/")
print("\n" + "="*60)
print("📊 최종 성능 요약")
print("="*60)
print("\n베이스라인 → 파인튜닝 후:")
print(f"  완전 일치: 10% → ??%")
print(f"  정규화 일치: 35% → ??%")
print(f"\n💡 위 평가 결과를 참고하세요!")
print("\n" + "="*60)
print("🚀 다음 단계")
print("="*60)
print("\n1️⃣ Google Drive에서 모델 다운로드")
print("2️⃣ 로컬(WSL)에서 추론 테스트")
print("3️⃣ 실제 서비스에 적용")
print("\n" + "="*60)

# HuggingFace Hub 업로드 (선택사항)
print("\n💡 HuggingFace Hub에 업로드하려면:")
print("   1. HuggingFace 토큰 준비")
print("   2. 아래 코드 주석 해제 후 실행")
print("\n")
print("# from huggingface_hub import login")
print("# login(token='your_token_here')")
print("# model.push_to_hub('your-username/deepseek-ocr-korean-signboard')")
print("# tokenizer.push_to_hub('your-username/deepseek-ocr-korean-signboard')")

print("\n\n🎊 축하합니다! 한국어 간판 OCR 모델 파인튜닝 완료! 🎊")